## Kernel `Fund-d3`

# Day 3 - Tutorial 1

## PART 1: Clustering Analysis

Clustering refers to grouping similar data points together, based on their attributes or features

This tutorial will be divided into two parts:

In the first part, we will explore various clustering methods using log data from a single well

In the second part, we will use the production dataset to generate a model that predicts the production trend for a well

### Step 1: Import Required libraries

In [0]:
# Import the requiered libraries

import pandas as pd
from pandas import DataFrame
from numpy import nan as NA
import matplotlib.pyplot as plt
import sys
import numpy as np

import dataiku
from dataiku import pandasutils as pdu

### Step 2: Import and Explore the Dataset

In [0]:
# Load the dataset 'w5' from Dataiku 

mydataset = dataiku.Dataset("w5")
w5_logs = mydataset.get_dataframe()

w5_logs

In [0]:
# Generate descriptive statistics of the log data




### Step 3: Preprocessing data -  Standardization

Clustering models are distance based algorithms, in order to measure similarities between observations and form clusters they use a distance metric. So, features with high ranges will have a bigger influence on the clustering. Therefore, it's a good practice to scale the data before applying clustering analysis.

Standardizing a dataset involves rescaling the distribution of values so that the mean of observed values is 0 and the standard deviation is 1.

In [0]:
# Use the StandardScaler() and scaler.transform() functions to standardize each column of the w5_logs dataset

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(w5_logs)


# The scaler.transform() function subtracts the mean and divides the result by the std of each column

w5_logs_s = scaler.transform(w5_logs)


# The return value after the transformation is a numpy array, let's explore it




In [0]:
# Convert the numpy array into a pandas DataFrame, call it 'w5_logs_scaled'

w5_logs_scaled = pd.DataFrame(w5_logs_s,
                              columns=['DEPTH','Pressure', 'AI', 'VpVs', 'SW', 'phid', 'vcl'])


In [0]:
# Print the standardized dataset and assign 'DEPTH' as index



### Step 4: Clustering Analysis Using K-Means

The KMeans algorithm clusters data by trying to separate samples in n groups of equal variance

In [0]:
# Extract 3 clusters of data using K-Means. Use the raw data (Not Scaled Data)

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)

pred = kmeans.fit_predict(w5_logs[['VpVs', 'AI']]) 


# Display the VpVs versus AI using the previously run K-means algorithm

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

plt.subplot(1, 2, 1)

ax1 = plt.scatter(w5_logs["AI"],w5_logs["VpVs"], color='gray', edgecolor='black', lw=0.3, s=30)
plt.xlabel("AI",fontsize=14)
plt.ylabel(r'$\frac{Vp}{Vs}$',fontsize=14)
plt.title('Raw Data',fontsize=14)


plt.subplot(1, 2, 2)

ax2 = plt.scatter(w5_logs["AI"],w5_logs["VpVs"], c=pred, edgecolor='black', lw=0.3, s=30,cmap="prism")
plt.xlabel("AI",fontsize=14)
plt.ylabel(r'$\frac{Vp}{Vs}$',fontsize=14)
plt.title('K-Means Data Not Scaled',fontsize=14)
plt.tight_layout()
plt.show()

In [0]:
# Extract 3 cluster of data using K-Means. Now let's use the scaled data

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)

pred = kmeans.fit_predict(w5_logs_scaled[['VpVs', 'AI']])


In [0]:
# Display the VpVs versus AI using the previously run K-means algorithm

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

plt.subplot(1, 2, 1)

ax1 = plt.scatter(w5_logs_scaled["AI"],w5_logs_scaled["VpVs"], color='gray', edgecolor='black', lw=0.3, s=30)
plt.xlabel("AI",fontsize=14)
plt.ylabel(r'$\frac{Vp}{Vs}$',fontsize=14)
plt.title('Raw Data',fontsize=14)


plt.subplot(1, 2, 2)

ax2 = plt.scatter(w5_logs_scaled["AI"],w5_logs_scaled["VpVs"], c=pred, edgecolor='black', lw=0.3, s=30,cmap="prism")
plt.xlabel("AI",fontsize=14)
plt.ylabel(r'$\frac{Vp}{Vs}$',fontsize=14)
plt.title('K-Means',fontsize=14)
plt.tight_layout()
plt.show()

### Step 5: Define a plotting function

Because we will be exploring several clustering methods which result requires the same plot, it's a good idea to define a plotting function

In [0]:
# Define a function to plot clustering results

def format_plot(title="",pred=""):
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

    plt.subplot(1, 2, 1)
    
    ax1 = plt.scatter(w5_logs_scaled["AI"],w5_logs_scaled["VpVs"], color='gray', edgecolor='black', lw=0.3, s=30)
    plt.xlabel("AI",fontsize=14)
    plt.ylabel(r'$\frac{Vp}{Vs}$',fontsize=14)
    plt.title('Raw Data',fontsize=14)


    plt.subplot(1, 2, 2)

    ax2 = plt.scatter(w5_logs_scaled["AI"],w5_logs_scaled["VpVs"], c=pred, edgecolor='black', lw=0.3, s=30,cmap="prism")
    plt.xlabel("AI",fontsize=14)
    plt.ylabel(r'$\frac{Vp}{Vs}$',fontsize=14)
    plt.title(title,fontsize=14)
    plt.tight_layout()      
    plt.show()

### Step 6: Identify the Optimun Number of Clusters - Elbow Method

In [0]:
# We can use the elbow plot to decide about the best number of classifications

from yellowbrick.cluster import KElbowVisualizer

model = KMeans()

elbow_vis = KElbowVisualizer(model, k=(2,11), size=(500, 400), locate_elbow=True, timings=False)
 

In [0]:
# Fit the data to the visualizer

elbow_vis.fit(w5_logs_scaled[['VpVs', 'AI']]) 

In [0]:
# Draw the elbow plot

elbow_vis.show()   

**Distortion**: It is calculated as the average of the squared distances from the cluster centers of the respective clusters. Typically, the Euclidean distance metric is used.

Good solutions aren’t those with the lowest score, but rather the ones where you notice a more or less abrupt discontinuity in the descent of the score (even just a change in the slope). 

A discontinuity means that the algorithm found some noticeable structural difference when looking for that number of
clusters in the data. In this example, a good solution seems to be four clusters

### Step 7: Clustering Analysis Using Gaussian Mixture Models

So instead of using a distance-based model (K-means), we will now use a distribution-based model.

In [0]:
# Cluster the log data using a Gaussian Mixture Model (GMM)

# First, train the gaussian model 

from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=5).fit(w5_logs_scaled[['VpVs', 'AI']])


# Generate predictions from the gaussian mixture model 

pred = gmm.predict(w5_logs_scaled[['VpVs', 'AI']])


# Plot the result using the format_plot function

format_plot("Gaussian Mixture Model",pred)


### Step 8: Clustering Analysis Using the Hierarchical Method

Hierarchical clustering is a general family of clustering algorithms that build nested clusters by merging or splitting them successively. 

The AgglomerativeClustering object performs a hierarchical clustering using a bottom up approach: each observation starts in its own cluster, and clusters are successively merged together. The linkage criteria determines the metric used for the merge strategy:

 - **Ward linkage** minimizes the sum of squared differences within all clusters

 - **Complete linkage** minimizes the maximum distance between observations of pairs of clusters
 
 - **Average linkage** minimizes the average of the distances between all observations of pairs of clusters


In [0]:
from sklearn.cluster import AgglomerativeClustering

# Perform clustering analysis using ward linkage criteria

ward = AgglomerativeClustering(n_clusters=3, linkage="ward")
clust_ward = ward.fit_predict(w5_logs_scaled[['VpVs', 'AI']])

# 'clust_ward' is a numpy array, explore it!


# Plot clustering results using the format_plot function

format_plot("Hierarchical- Ward Linkage", clust_ward)


In [0]:
# Perform clustering analysis using complete linkage criteria

complete = AgglomerativeClustering(n_clusters=3, linkage="complete")
clust_complete= complete.fit_predict(w5_logs_scaled[['VpVs', 'AI']])

# 'clust_complete' is a numpy array, explore it!


# Plot clustering results using the format_plot function




In [0]:
# Perform clustering analysis using average linkage criteria



# 'clust_average' is a numpy array, explore it!


# Plot clustering results using the format_plot function




### Step 9: Clustering Analysis Using DBSCAN Method

DBSCAN - Density-Based Spatial Clustering of Applications with Noise (i.e., outliers). 

The main idea behind DBSCAN is that a data value belongs to a cluster if it is close to many data values from that cluster

In [0]:
from sklearn import cluster

# Train the dbscan model

dbscan = cluster.DBSCAN(eps=0.09, min_samples=3)


# Generate predictions using the dbscan model 

db_pred = dbscan.fit_predict(w5_logs_scaled[['VpVs', 'AI']])


# Plot clustering results using the format_plot function




In [0]:
# Note that DBSCAN does not require specifying the number of clusters, let's review it ('db_pred')


# Noisy data points are given the label -1. They do not belong to any cluster

## Part 2: Prediction of Production Patterns Using Clustering Analysis

In the second part of this tutorial, we will learn how to train a clustering model using the production data from four wells, and then use this model to predict the production trends from another well

In [0]:
# Read and display the production data from Dataiku

mydataset = dataiku.Dataset("production_data")
production = mydataset.get_dataframe()

# Make sure that the column 'Prod Date' is treated as date

production["Prod Date"]= pd.to_datetime(production["Prod Date"])

production

In [0]:
# Set the column 'Prod Date' as index



In [0]:
# Print the production data to visualize changes



In [0]:
# Investigate the total number of wells in the dataset



In [0]:
# Create a new dataframe that contains only the production data from well 'w1', call it 'production_test'



In [0]:
# Create a new dataframe that contains the production data from the rest of the wells ('w2', 'w3' and 'w4'), call it 'production_train'

production_train= production.query('Unformatted_UWI!="w1"')



In [0]:
# Verify the wells contained in each dataframe ('production_test' & 'production_train')



In [0]:
# Print the production data to visualize changes



In [0]:
# Now let's plot the monthly gas production versus date for the train wells  

production_train.groupby("Unformatted_UWI")["Monthly Gas (E3m3)"].plot(marker='.', markersize=5, 
                                                                       legend=True, figsize=(8, 6),
                                                         title='Monthly Gas Production - Train wells')

plt.xlabel('Production date')
plt.ylabel('Monthly Gas Production (e3m3)')
plt.show()

# Note that the production from the three wells begins at different times

In [0]:
# Plot the entire production data without discretizing it by well 

production_train.reset_index()["Monthly Gas (E3m3)"].plot(marker='.', markersize=5, 
                                                                       legend=True, figsize=(8, 6),
                                                         title='Monthly Gas Production - Train wells | Combined')

plt.xlabel('Month')
plt.ylabel('Monthly Gas Production (e3m3)')
plt.show()

# Note that the "production peaks" correspond to the order in which the wells appear in the dataset (w3 -> w2 -> w4) 

Now that we have combined the production patterns from the train wells, the next step will be to train a model to see if we can separate different parts of the time series using another feature. For this, we can create a new feature using the .diff() function.

.diff() function is used to calculate the difference between values in an array along with a given axis

In [0]:
# Create a function to add a new feature to the production dataset, call it 'preprocess_data'

def preprocess_data(df):
    d= df.reset_index()[["Monthly Gas (E3m3)"]]
    d["diff"]=d.diff()
    d.dropna(inplace=True)
    return d

In [0]:
# Apply the 'preprocess_data' function to the production test and train dataframes

production_train_processed= preprocess_data(production_train)

production_test_processed= preprocess_data(production_test)

In [0]:
# Display the resulting train dataframe 'production_train_processed'



In [0]:
# Display the resulting test dataframe 'production_test_processed'



In [0]:
# Plot the production and diff variables in the same graph

production_train_processed.plot(marker='.', markersize=5, legend=True, figsize=(8, 6),
                                                         title='Gas Production & diff - Train wells | Combined')

plt.xlabel('Month')
plt.show()


In [0]:
# Use the elbow plot on the train dataset to identify the optimum number of clusters

from yellowbrick.cluster import KElbowVisualizer

model = KMeans(random_state=42)
elb_visualizer = KElbowVisualizer(model, k=(2,11), size=(600, 400), locate_elbow=True, timings=False)
elb_visualizer.fit(production_train_processed)
elb_visualizer.show()


In [0]:
# Train a K-means clustering model using the production data on the train set (w2, w3, w4)

d=production_train_processed
kmeans = KMeans(n_clusters=3)
prod_train_pred = kmeans.fit_predict(d)

# 'prod_train_pred' is a numpy array, explore it!



In [0]:
# Plot the results of the clustering on the train set (w2, w3, w4)

plt.figure(figsize=(8, 6));
plt.scatter(d.index, d["Monthly Gas (E3m3)"], label = 'prod_train_pred', 
            c=prod_train_pred, edgecolor='black', lw=0.5, s=50, cmap=plt.get_cmap('prism'))

plt.title('Production trends | Train wells')
plt.xlabel('Month')
plt.ylabel('Monthly Gas Production (e3m3)')
plt.legend(prod_train_pred) 
plt.show()

In [0]:
# Implement the previously trained K-means model to predit the production trend on the test set (w1)

d=production_test_processed
prod_test_pred = kmeans.predict(d)


# Plot the results of the clustering on the test set (w1)

plt.figure(figsize=(8, 6));
plt.scatter(d.index, d["Monthly Gas (E3m3)"], linestyle='solid', marker="o",  label = 'prod_train_pred', 
            c=prod_test_pred, edgecolor='black', lw=0.5, s=50, cmap=plt.get_cmap('prism'))

plt.title('W1 - Predicted Production')
plt.xlabel('Month')
plt.ylabel('Monthly Gas Production (e3m3)')
plt.show()

In [0]:
# Now let's plot the actual production data for well 'w1' and compare it with the predicted production

production_test.plot(marker='o', markersize= 7, 
                     legend=True, figsize=(8, 6),
                     title='W1 - Actual Production')

plt.xlabel('Production date')
plt.ylabel('Monthly Gas Production (e3m3)')
plt.show()